In [ ]:
#pip install PEPit

# Readme

This notebook exemplifies the results of [1, Section 3.2] (and in particular those of [1, Proposition 3.6]) for the numerical analysis of cyclic and randomized block-coordinate descent algorithms for convex optimization.
(the improvements in terms of performance guarantees is extremely small, but exist)

> [1] Rubbens, Anne, Hendrickx, Julien M., and Taylor, Adrien B. "A constructive approach to strengthen algebraic descriptions of function and operator classes."

The code requires the installation of the [PEPit](https://pepit.readthedocs.io/en/latest/) package, e.g., by uncommenting the pip install command above.

# Function class: refined characterization of convex functions that are smooth by block.

In [1]:
import numpy as np
from PEPit import PEP
from PEPit.function import Function
from PEPit.block_partition import BlockPartition

from PEPit.functions import BlockSmoothConvexFunction

In [3]:
class RefinedBlockSmoothConvexFunction(Function):
    def __init__(self,
                 partition,
                 L,
                 lam=[0.],
                 is_leaf=True,
                 decomposition_dict=None,
                 reuse_gradient=True,
                 name=None):
        
        super().__init__(is_leaf=is_leaf,
                         decomposition_dict=decomposition_dict,
                         reuse_gradient=True,
                         name=name,
                         )

        # Store partition and L
        assert isinstance(partition, BlockPartition)
        if partition.get_nb_blocks() > 1:
            assert isinstance(lam, list)
            assert isinstance(L, list)
            assert len(L) == partition.get_nb_blocks()
            for Li in L:
                assert Li < np.inf
            for lami in lam:
                assert lami <= 1/2
                assert lami >= 0

        self.partition = partition
        self.lam = lam
        self.L = L

    def add_class_constraints(self):
        """
        Formulates the list of necessary constraints for interpolation for self (block smooth convex function);
        see [1, Proposition 7].

        """
        # Set function ID
        function_id = self.get_name()
        if function_id is None:
            function_id = "Function_{}".format(self.counter)

        # Set tables_of_constraints attributes
        for m in range(self.partition.get_nb_blocks()):
            for l in range(self.partition.get_nb_blocks()):
                self.tables_of_constraints["smoothness_convexity_block_{}{}".format(m, l)] = [[[]]]*len(self.list_of_points)

        # Browse list of points and create interpolation constraints
        for i, point_i in enumerate(self.list_of_points):

            xi, gi, fi = point_i
            xi_id = xi.get_name()
            if xi_id is None:
                xi_id = "Point_{}".format(i)

            for j, point_j in enumerate(self.list_of_points):

                xj, gj, fj = point_j
                xj_id = xj.get_name()
                if xj_id is None:
                    xj_id = "Point_{}".format(j)
                
                for k, point_k in enumerate(self.list_of_points):
                    xk, gk, fk = point_k
                    xk_id = xk.get_name()
                    if xk_id is None:
                        xk_id = "Point_{}".format(k)
                        
                    if not (point_i == point_j and point_i == point_k):
                        for lam in self.lam:
                            for m in range(self.partition.get_nb_blocks()):
                                for l in range(self.partition.get_nb_blocks()):
                                    # partial gradients for block m
                                    gim = self.partition.get_block(gi, m)
                                    gjm = self.partition.get_block(gj, m)
                                    gkm = self.partition.get_block(gk, m)
                                    
                                    # partial gradients for block l
                                    gjl = self.partition.get_block(gj, l)
                                    gkl = self.partition.get_block(gk, l)
                                    
                                    # Necessary conditions for interpolation
                                    constraint = (0 >= (1-lam) * (-fi + fj + gj * (xi - xj) + 1 / (2 * self.L[m]) * (gim - gjm) ** 2) +
                                                  lam * (-fi + fk + gk * (xi - xk) + 1/(2 * self.L[m]) * (gim - gkm) ** 2) +
                                                  lam * (1-lam)  * ( 1 / (2 * self.L[l]) * (gjl - gkl) ** 2 - 1 / (2 * self.L[m]) * (gjm - gkm) ** 2))
                                    
                                    constraint.set_name("IC_{}_smoothness_convexity_block_{}{}({}, {}, {})".format(function_id, m, l, xi_id, xj_id, xk_id))
                                    self.tables_of_constraints["smoothness_convexity_block_{}{}".format(m, l)][i].append(constraint)
                                    self.list_of_class_constraints.append(constraint)

## Example 1: cyclic block-coordinate descent

In [4]:
# for comparison
import PEPit.examples.unconstrained_convex_minimization.cyclic_coordinate_descent as CCD

def wc_cyclic_coordinate_descent_refined(L, n, refinment_param=[.375], wrapper="cvxpy", solver=None, verbose=1):

    # Instantiate PEP
    problem = PEP()

    # Declare a partition of the ambient space in d blocks of variables
    d = len(L)
    partition = problem.declare_block_partition(d=d)

    # Declare a strongly convex smooth function
    func = problem.declare_function(RefinedBlockSmoothConvexFunction, L=L, lam=refinment_param, partition=partition)

    # Start by defining its unique optimal point xs = x_* and corresponding function value fs = f_*
    xs = func.stationary_point()
    fs = func(xs)

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()

    # Set the initial constraint that is the distance between x0 and x^*
    problem.set_initial_condition((x0 - xs) ** 2 <= 1)

    # Run n steps of the GD method
    x = x0
    for k in range(n):
        i = k % d
        x = x - 1 / L[i] * partition.get_block(func.gradient(x), i)

    # Set the performance metric to the function values accuracy
    problem.set_performance_metric(func(x) - fs)

    # Solve the PEP
    pepit_verbose = max(verbose, 0)
    pepit_tau = problem.solve(wrapper=wrapper, solver=solver, verbose=pepit_verbose)

    # Compute theoretical guarantee (for comparison)
    theoretical_tau = None

    # Print conclusion if required
    if verbose != -1:
        print('*** Example file: worst-case performance of cyclic coordinate descent with fixed step-sizes ***')
        print('\tPEPit guarantee:\t f(x_n)-f_* <= {:.6} ||x_0 - x_*||^2'.format(pepit_tau))

    # Return the worst-case guarantee of the evaluated method (and the reference theoretical value)
    return pepit_tau, theoretical_tau

In [5]:
L = [1,1]
n = 2
verbose = -1
refinment_param = [.375]
pepit_tau_refined, _ = wc_cyclic_coordinate_descent_refined( L, n, refinment_param, verbose = verbose )
pepit_tau, _ = CCD.wc_cyclic_coordinate_descent( L, n, verbose = verbose )

print('*** Cyclic coordinate descent *** ')
print('\tPEPit guarantee (std inequalities):\t f(x_n)-f_* <= {:.6} ||x_0 - x_*||^2'.format(pepit_tau))
print('\tPEPit guarantee (refined inequalities):\t f(x_n)-f_* <= {:.6} ||x_0 - x_*||^2'.format(pepit_tau_refined))
print('\tIs the refined inequality stronger? {}'.format(pepit_tau_refined<=pepit_tau))

*** Cyclic coordinate descent *** 
	PEPit guarantee (std inequalities):	 f(x_n)-f_* <= 0.22515 ||x_0 - x_*||^2
	PEPit guarantee (refined inequalities):	 f(x_n)-f_* <= 0.222741 ||x_0 - x_*||^2
	Is the refined inequality stronger? True


## Example 2: randomized block-coordinate descent

Let $A_t\geq 0$. We compute the largest possible $A_{t+1}$ such that the inequality 

$$ E_{i_k} \left[ A_{t+1}  (f(x^{(i_k)}_{k+1})-f(x_\star)) + \tfrac12 \|x_{k+1}^{(i_k)} -x_\star\|_{\{L_i\}}^2 \right]\leqslant A_t(f(x_{k})-f(x_\star)) + \tfrac12 \|x_{k} -x_\star\|_{\{L_i\}}^2 $$

is valid for all functions that are convex and smooth by blocks, and for all points $x_{k+1},x_k,x_\star$ ( such that $x_\star$ is a minimizer of $f$ and such that $x_{k+1}^{(i_k)}$ is obtained by RBCD updating the $i_k$th block (with stepsize $1/L_i$)).

In [6]:
def wc_randomized_coordinate_descent_smooth_convex(L, gamma, d, At, wrapper="mosek", solver=None, verbose=1):
    # Instantiate PEP
    problem = PEP()

    # Declare a partition of the ambient space in d blocks of variables
    partition = problem.declare_block_partition(d=d)

    # Declare a smooth convex function by blocks
    func = problem.declare_function(BlockSmoothConvexFunction, L=L, partition=partition )

    # Start by defining its unique optimal point xs = x_* and corresponding function value fs = f_*
    xs = func.stationary_point()
    fs = func(xs)

    # Then define the point x_{t-1} of the algorithm
    xt_minus_1 = problem.set_initial_point()

    # Define the Lyapunov function
    def flyap(x):
        fl = (func(x) - fs)
        return  fl
    def distlyap(x):
        distl = 0 * (func(x) - fs)
        for i in range(d):
            distl = distl + L[i] / 2 * (partition.get_block(x - xs,i)) ** 2
        return distl

    # Compute all the possible outcomes of the randomized coordinate descent step
    gt_minus_1 = func.gradient(xt_minus_1)
    xt_list = [xt_minus_1 - gamma[i] * partition.get_block(gt_minus_1, i) for i in range(d)]

    # Compute the expected value of the Lyapunov from the different possible outcomes
    func_t = np.mean([flyap(xt) for xt in xt_list])
    dist_t = np.mean([distlyap(xt) for xt in xt_list])

    # Set the initial condition
    problem.set_initial_condition( At * flyap(xt_minus_1) + distlyap(xt_minus_1) - dist_t <= 1 )

    # Set the performance metric to the variance
    problem.set_performance_metric(func_t)

    # Solve the PEP
    pepit_verbose = max(verbose, 0)
    pepit_tau = problem.solve(wrapper=wrapper, solver=solver, verbose=pepit_verbose)

    # Return the worst-case guarantee of the evaluated method (and the reference theoretical value)
    return 1/pepit_tau

def wc_randomized_coordinate_descent_smooth_convex_refined(L, gamma, d, At, refinment_param, wrapper="mosek", solver=None, verbose=1):
    # Instantiate PEP
    problem = PEP()

    # Declare a partition of the ambient space in d blocks of variables
    partition = problem.declare_block_partition(d=d)

    # Declare a smooth convex function by blocks
    func = problem.declare_function(RefinedBlockSmoothConvexFunction, L=L, lam=refinment_param, partition=partition )

    # Start by defining its unique optimal point xs = x_* and corresponding function value fs = f_*
    xs = func.stationary_point()
    fs = func(xs)

    # Then define the point x_{t-1} of the algorithm
    xt_minus_1 = problem.set_initial_point()

    # Define the Lyapunov function
    def flyap(x):
        fl = (func(x) - fs)
        return  fl
    def distlyap(x):
        distl = 0 * (func(x) - fs)
        for i in range(d):
            distl = distl + L[i] / 2 * (partition.get_block(x - xs,i)) ** 2
        return distl

    # Compute all the possible outcomes of the randomized coordinate descent step
    gt_minus_1 = func.gradient(xt_minus_1)
    xt_list = [xt_minus_1 - gamma[i] * partition.get_block(gt_minus_1, i) for i in range(d)]

    # Compute the expected value of the Lyapunov from the different possible outcomes
    func_t = np.mean([flyap(xt) for xt in xt_list])
    dist_t = np.mean([distlyap(xt) for xt in xt_list])

    # Set the initial condition
    problem.set_initial_condition( At * flyap(xt_minus_1) + distlyap(xt_minus_1) - dist_t <= 1 )

    # Set the performance metric to the variance
    problem.set_performance_metric(func_t)

    # Solve the PEP
    pepit_verbose = max(verbose, 0)
    pepit_tau = problem.solve(wrapper=wrapper, solver=solver, verbose=pepit_verbose)

    # Return the worst-case guarantee of the evaluated method (and the reference theoretical value)
    return 1/pepit_tau


In [7]:
L = [2.,.5]
d = len(L)
gamma=1./np.array(L)
At = 10
verbose = -1
refinment_param = [.375]
At1_refined = wc_randomized_coordinate_descent_smooth_convex_refined(L, gamma, d, At, refinment_param, verbose=verbose)
At1 = wc_randomized_coordinate_descent_smooth_convex(L, gamma, d, At, verbose=verbose)

print('*** Cyclic coordinate descent *** ')
print('\tPEPit guarantee (std inequalities):\t A_(t+1)={}'.format(At1))
print('\tPEPit guarantee (refined inequalities):\t A_(t+1)={}'.format(At1_refined))
print('\tIs the refined inequality stronger (larger A_(t+1))? {}'.format(At1_refined>=At1))
print('\tNote: the difference between the bounds is typically very small and might disappear as the problem becomes larger (e.g., larger d) due to numerical inaccuracies.')

*** Cyclic coordinate descent *** 
	PEPit guarantee (std inequalities):	 A_(t+1)=10.50076493496037
	PEPit guarantee (refined inequalities):	 A_(t+1)=10.502234560108477
	Is the refined inequality stronger (larger A_(t+1))? True
	Note: the difference between the bounds is typically very small and might disappear as the problem becomes larger (e.g., larger d) due to numerical inaccuracies.
